In [20]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [21]:
ex1 = pd.read_parquet('train/5.잔액정보/2018_잔액정보_전체.parquet')

In [22]:
member_df = pd.read_parquet('train/1.회원정보/2018_회원정보_전체.parquet')
member_seg = member_df[['ID', 'Segment']].drop_duplicates(subset='ID')
ex1 = ex1.merge(member_seg, on='ID', how='left')



In [23]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '잔액_일시불_B0M', '잔액_할부_B0M', '잔액_현금서비스_B0M', '잔액_리볼빙일시불이월_B0M', '잔액_리볼빙CA이월_B0M', '잔액_카드론_B0M', '월중평잔_일시불_B0M', '월중평잔_할부_B0M', '월중평잔_CA_B0M', '카드론잔액_최종경과월', '연체일자_B0M', '연체잔액_B0M', '연체잔액_일시불_B0M', '연체잔액_할부_B0M', '연체잔액_현금서비스_B0M', '연체잔액_카드론_B0M', '연체잔액_대환론_B0M', '잔액_현금서비스_B1M', '잔액_현금서비스_B2M', '잔액_카드론_B1M', '잔액_카드론_B2M', '잔액_카드론_B3M', '잔액_카드론_B4M', '잔액_카드론_B5M', '잔액_할부_B1M', '잔액_할부_B2M', '잔액_일시불_B1M', '잔액_일시불_B2M', '연체일수_B1M', '연체일수_B2M', '연체원금_B1M', '연체원금_B2M', '연체일수_최근', '연체원금_최근', '최종연체회차', '매각잔액_B1M', '최종연체개월수_R15M', 'RV_평균잔액_R12M', 'RV_최대잔액_R12M', 'RV_평균잔액_R6M', 'RV_최대잔액_R6M', 'RV_평균잔액_R3M', 'RV_최대잔액_R3M', 'RV잔액이월횟수_R6M', 'RV잔액이월횟수_R3M', '잔액_할부_유이자_B0M', '잔액_할부_무이자_B0M', '잔액_할부_해외_B0M', '연체잔액_일시불_해외_B0M', '연체잔액_RV일시불_B0M', '연체잔액_RV일시불_해외_B0M', '연체잔액_할부_해외_B0M', '연체잔액_CA_B0M', '연체잔액_CA_해외_B0M', '월중평잔', '월중평잔_일시불', '월중평잔_RV일시불', '월중평잔_할부', '월중평잔_CA', '월중평잔_카드론', '평잔_3M', '평잔_일시불_3M', '평잔_일시불_해외_3M', '평잔_RV일시불_3M', '평잔_RV일시불_해외_3M', '평잔_할부_3M', '평잔_할부_해외_3M', '평

In [24]:
ex1 = ex1.fillna(-1)

In [25]:
num_df = ex1.select_dtypes(include=[np.number]).dropna()
low_var = num_df.var()[ num_df.var() < 0.01 ].index.tolist()
low_var

['카드론잔액_최종경과월',
 '최종연체개월수_R15M',
 'RV잔액이월횟수_R6M',
 'RV잔액이월횟수_R3M',
 '연체잔액_일시불_해외_B0M',
 '연체잔액_RV일시불_해외_B0M',
 '연체잔액_할부_해외_B0M',
 '연체잔액_CA_해외_B0M']

In [26]:
cols_to_drop = ['카드론잔액_최종경과월',
 '최종연체개월수_R15M',
 'RV잔액이월횟수_R6M',
 'RV잔액이월횟수_R3M',
 '연체잔액_일시불_해외_B0M',
 '연체잔액_RV일시불_해외_B0M',
 '연체잔액_할부_해외_B0M',
 '연체잔액_CA_해외_B0M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [27]:
missing_mask = ex1.isna() | (ex1 == -1)
missing_ratio = missing_mask.mean()

high_na = missing_ratio[missing_ratio > 0.2].index.tolist()

high_const_cols = []
threshold_const = 0.8

for col in ex1.columns:
    top_ratio = ex1[col].value_counts(normalize=True, dropna=False).values[0]
    if top_ratio > threshold_const:
        high_const_cols.append(col)

to_drop = list(set(high_na + high_const_cols))

if 'Segment' in to_drop:
    to_drop.remove('Segment')

print("삭제 대상 컬럼 (결측>20% 또는 동일값>80%):", to_drop)

ex1.drop(columns=to_drop, inplace=True)


삭제 대상 컬럼 (결측>20% 또는 동일값>80%): ['RV_평균잔액_R3M', '연체일수_B2M', '평잔_CA_해외_6M', 'RV_최대잔액_R3M', '연체일자_B0M', '잔액_카드론_B1M', '월중평잔_CA', '잔액_카드론_B3M', '평잔_RV일시불_3M', '월중평잔_카드론', '연체잔액_일시불_B0M', '월중평잔_CA_B0M', 'RV_평균잔액_R12M', '평잔_CA_6M', '잔액_현금서비스_B1M', '잔액_리볼빙CA이월_B0M', '평잔_할부_해외_3M', '평잔_일시불_해외_6M', '평잔_CA_해외_3M', '매각잔액_B1M', '잔액_할부_유이자_B0M', 'RV_최대잔액_R12M', '잔액_현금서비스_B2M', '잔액_현금서비스_B0M', '연체잔액_현금서비스_B0M', '연체잔액_CA_B0M', '연체잔액_할부_B0M', 'RV_평균잔액_R6M', '평잔_RV일시불_해외_6M', '잔액_카드론_B0M', '평잔_CA_3M', '연체잔액_대환론_B0M', '잔액_할부_해외_B0M', '평잔_일시불_해외_3M', '연체원금_최근', '연체잔액_RV일시불_B0M', '연체일수_B1M', '평잔_카드론_6M', '연체잔액_카드론_B0M', '연체잔액_B0M', '잔액_카드론_B5M', '평잔_할부_해외_6M', '평잔_RV일시불_해외_3M', '평잔_RV일시불_6M', '연체원금_B1M', '잔액_카드론_B4M', '월중평잔_RV일시불', '연체원금_B2M', 'RV_최대잔액_R6M', '잔액_카드론_B2M', '잔액_리볼빙일시불이월_B0M', '평잔_카드론_3M', '연체일수_최근']


In [28]:
ex1

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,잔액_할부_B1M,잔액_할부_B2M,잔액_일시불_B1M,잔액_일시불_B2M,...,월중평잔,월중평잔_일시불,월중평잔_할부,평잔_3M,평잔_일시불_3M,평잔_할부_3M,평잔_6M,평잔_일시불_6M,평잔_할부_6M,Segment
0,201807,TRAIN_000000,998,962,1084,547,640,805,736,1083,...,17237,1503,532,18557,1791,601,15988,2440,572,D
1,201807,TRAIN_000001,2565,2390,4090,2553,2779,2699,6795,1424,...,7967,4447,2639,7031,3761,2886,7045,2677,2736,E
2,201807,TRAIN_000002,5312,5113,5006,8778,9451,4233,5150,6143,...,59917,5540,8903,70265,6796,7758,66549,9118,4429,C
3,201807,TRAIN_000003,730,5025,487,5607,6134,3719,462,708,...,27854,606,5598,33741,772,4931,30139,884,5097,D
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,28,21,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,3351,0,4412,0,0,0,4142,4362,...,29429,5515,0,35344,9424,0,67826,12524,0,D
2399997,201812,TRAIN_399997,2524,2960,2694,3374,3291,3474,1523,2898,...,7383,3046,3297,8500,2998,2833,8627,3241,3995,C
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [29]:
import pandas as pd
import numpy as np

num_df = ex1.select_dtypes(include=[np.number]).dropna()

corr = num_df.corr().abs()

high_corr_pairs = (
    corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
        .stack()
        .reset_index()
)
high_corr_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']
high_corr_pairs = high_corr_pairs[high_corr_pairs['Correlation'] > 0.7]

if not np.issubdtype(ex1['Segment'].dtype, np.number):
    segment_map = {label: idx for idx, label in enumerate(sorted(ex1['Segment'].unique()))}
    ex1['Segment_encoded'] = ex1['Segment'].map(segment_map)
else:
    ex1['Segment_encoded'] = ex1['Segment']

segment_corr = ex1[num_df.columns].corrwith(ex1['Segment_encoded']).abs()

high_corr_pairs['Corr_with_Segment_1'] = high_corr_pairs['Feature_1'].map(segment_corr)
high_corr_pairs['Corr_with_Segment_2'] = high_corr_pairs['Feature_2'].map(segment_corr)

high_corr_pairs = high_corr_pairs.sort_values(by='Correlation', ascending=False).reset_index(drop=True)

print(f"▶ 상관계수 0.7 초과 변수쌍 수: {len(high_corr_pairs)}")
display(high_corr_pairs)


▶ 상관계수 0.7 초과 변수쌍 수: 52


,Feature_1,Feature_2,Correlation,Corr_with_Segment_1,Corr_with_Segment_2
0,월중평잔_할부_B0M,월중평잔_할부,0.993529,0.234879,0.232085
1,월중평잔_할부_B0M,잔액_할부_B1M,0.992217,0.234879,0.228537
2,잔액_할부_B1M,월중평잔_할부,0.988888,0.228537,0.232085
3,잔액_할부_B1M,평잔_할부_3M,0.975334,0.228537,0.233328
4,월중평잔_할부_B0M,평잔_할부_3M,0.972890,0.234879,0.233328
5,월중평잔_할부,평잔_할부_3M,0.971621,0.232085,0.233328
6,월중평잔_일시불_B0M,월중평잔_일시불,0.966031,0.444621,0.441527
7,평잔_할부_3M,평잔_할부_6M,0.961928,0.233328,0.250312
8,잔액_할부_B2M,평잔_할부_3M,0.961725,0.228411,0.233328
9,월중평잔_일시불,평잔_일시불_3M,0.955870,0.441527,0.434514


In [30]:
to_drop = []

for _, row in high_corr_pairs.iterrows():
    f1, f2 = row['Feature_1'], row['Feature_2']
    c1, c2 = row['Corr_with_Segment_1'], row['Corr_with_Segment_2']
    
    if pd.isna(c1) or pd.isna(c2):
        continue
    
    if c1 < c2:
        to_drop.append(f1)
    else:
        to_drop.append(f2)

to_drop = list(set(to_drop))

# 결과 출력
print(f"▶ 제거 대상 피처 수: {len(to_drop)}")
print("제거할 피처 목록:")
print(to_drop)

▶ 제거 대상 피처 수: 15
제거할 피처 목록:
['잔액_일시불_B0M', '잔액_일시불_B2M', '평잔_할부_6M', '평잔_3M', '월중평잔_할부_B0M', '잔액_할부_B1M', '월중평잔_일시불', '월중평잔_할부', '평잔_일시불_3M', '잔액_할부_B2M', '잔액_일시불_B1M', '월중평잔_일시불_B0M', '잔액_할부_무이자_B0M', '평잔_6M', '평잔_할부_3M']


In [31]:
cols_to_drop = ['잔액_일시불_B0M', '잔액_일시불_B2M', '평잔_할부_6M', '평잔_3M', '월중평잔_할부_B0M', '잔액_할부_B1M', '월중평잔_일시불', '월중평잔_할부', '평잔_일시불_3M', '잔액_할부_B2M', '잔액_일시불_B1M', '월중평잔_일시불_B0M', '잔액_할부_무이자_B0M', '평잔_6M', '평잔_할부_3M']
ex1.drop(columns=cols_to_drop, inplace=True)

In [33]:
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

num_df = (
    ex1
    .select_dtypes(include=[np.number])
    .drop(columns=['기준년월'], errors='ignore')
    .dropna()
)


X = add_constant(num_df)

vif_df = pd.DataFrame({
    'feature': X.columns,
    'VIF': [variance_inflation_factor(X.values, i)
            for i in range(X.shape[1])]
})

vif_df = vif_df[vif_df['feature'] != 'const'].reset_index(drop=True)

high_vif = vif_df[vif_df['VIF'] > 10]

print("VIF > 10인 컬럼들:", high_vif['feature'].tolist())
print(high_vif)


VIF > 10인 컬럼들: []
Empty DataFrame
Columns: [feature, VIF]
Index: []


In [34]:
from sklearn.preprocessing import LabelEncoder

df_corr = ex1.drop(columns=['기준년월', 'ID']).copy()

le = LabelEncoder()
df_corr['Segment_encoded'] = le.fit_transform(df_corr['Segment'])

correlations = df_corr.corr(numeric_only=True)['Segment_encoded'].drop('Segment_encoded')

selected_cols = correlations[correlations.abs() > 0.1].index.tolist()

final_cols = ['기준년월', 'ID','Segment'] + selected_cols

ex1_filtered = ex1[final_cols]


In [35]:
cols = ex1_filtered.columns.tolist()
cols

['기준년월', 'ID', 'Segment', '잔액_할부_B0M', '월중평잔', '평잔_일시불_6M']

In [36]:
ex1_filtered.to_parquet('잔액_전처리_Segment.parquet', index=False)